In [31]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

from time import time                                # Record computation time
from numpy.fft import fft, ifft, fftshift, ifftshift # Fast Fourier Transform
from scipy.interpolate import interp1d               # Interpolation
from functools import partial                        # Function wrapper

plt.rcParams['axes.axisbelow'] = True   # Set axes and grid elements to be below the figure

In [32]:
N_SIMS = 20000                           # FIXME: number of simulations
N_BLOCKS = 100
N_STEPS = 200                            # FIXME: number of steps (excluding t = 0)
T = 1                                    # FIXME: time horizon
S0 = 100                                 # initial stock price (not using here for simplicity)

dt = T / N_STEPS                         # time interval

# GBM parameters
K = 100
R = 0.1                                 # FIXME: risk-free interest rate
Q = 0.00
MU = R - Q                               # FIXME: drift coefficient (GBM)
SIGMA = 0.2                              # FIXME: diffusion coefficient


# Jump parameters (NCPP normal cumulative Possion process)
LAMBDA = 0.8                             # FIXME: jump rate
MU_J = 0.0                              # FIXME: jump mean
SIGMA_J = 0.5                           # FIXME: jump standard deviation

m = LAMBDA * (np.exp(MU_J + 0.5 * SIGMA_J**2) - 1) # Martingale correction

# Fourier parameters ---------------------------------------------------------------------------------------------
X_WIDTH = 6                                # TODO: Width of the interval
N_GRIDS = 2**8                             # TODO: Number of grid points
ALPHA = -10                                # TODO: Dampening factor for a call


In [33]:
def fft_Lewis(K, S0, cf, interp="cubic"):
    """
    K = vector of strike
    S = spot price scalar
    cf = characteristic function
    interp can be cubic or linear
    """
    N = 2**12  # FFT more efficient for N power of 2
    B = 200  # integration limit
    dx = B / N
    x = np.arange(N) * dx  # the final value B is excluded

    weight = np.arange(N)  # Simpson weights
    weight = 3 + (-1) ** (weight + 1)
    weight[0] = 1
    weight[N - 1] = 1

    dk = 2 * np.pi / B
    b = N * dk / 2
    ks = -b + dk * np.arange(N)

    integrand = np.exp(-1j * b * np.arange(N) * dx) * cf(x - 0.5j) * 1 / (x**2 + 0.25) * weight * dx / 3
    integral_value = np.real(ifft(integrand) * N)

    if interp == "linear":
        spline_lin = interp1d(ks, integral_value, kind="linear")
        prices = S0 - np.sqrt(S0 * K) * np.exp(-R * T) / np.pi * spline_lin(np.log(S0 / K))
    elif interp == "cubic":
        spline_cub = interp1d(ks, integral_value, kind="cubic")
        prices = S0 - np.sqrt(S0 * K) * np.exp(-R * T) / np.pi * spline_cub(np.log(S0 / K))
    return prices

# characteristic function of the Merton process at time t
def cf_mert(u, t=1, mu=1, sig=2, lam=0.8, muJ=0, sigJ=0.5):
    return np.exp(
        t * (1j * u * mu - 0.5 * u**2 * sig**2 + lam * (np.exp(1j * u * muJ - 0.5 * u**2 * sigJ**2) - 1))
    )

cf_mert_wrapped = partial(cf_mert, t=T, mu = R - Q - 0.5 * SIGMA**2 - m, sig = SIGMA, lam  =LAMBDA, muJ = MU_J, sigJ = SIGMA_J)

prices_BS_cub = fft_Lewis(K, S0, cf_mert_wrapped, interp="cubic")
prices_BS_lin = fft_Lewis(K, S0, cf_mert_wrapped, interp="linear")

print(prices_BS_cub)
print(prices_BS_lin)

call_fourier = prices_BS_cub
# Pull call parity with dividend
put_fourier = call_fourier - S0 + K * np.exp(-R * T)

22.016367621906383
22.016367621906383


In [34]:
# # # FOURIER SOLUTION ===============================================================================================
# temp = time()

# # Controls


# def payoff(x, xi, ALPHA, K, L, U, C, theta):
#     # Scale
#     S = C * np.exp(x)

#     # Payoff; see e.g. Green, Fusai, Abrahams 2010, Eq. (3.24)
#     g = np.exp(ALPHA * x) * np.maximum(theta * (S - K), 0) * (S >= L) * (S <= U)

#     # Analytical Fourier transform of the payoff
#     l = np.log(L / C)  # lower log barrier
#     k = np.log(K / C)  # log strike
#     u = np.log(U / C)  # upper log barrier

#     # Integration bounds
#     if theta == 1:  # call
#         a = max(l, k)
#         b = u
#     else:  # put
#         a = min(k, u)
#         b = l

#     # Green, Fusai, Abrahams 2010 Eq. (3.26) with extension to put option
#     xi2 = ALPHA + 1j * xi
#     G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2) - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)

#     # Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
#     if ALPHA == 0:
#         G[len(G) // 2] = C * (np.exp(b) - np.exp(a) - np.exp(k) * (b - a))
#     elif ALPHA == -1:
#         G[len(G) // 2] = C * (b - a + np.exp(k - b) - np.exp(k - a))

#     return S, g, G

# # Grids in real and Fourier space
# N = N_GRIDS // 2
# b = X_WIDTH / 2  # upper bound of the support in real space
# dx = X_WIDTH / N_GRIDS  # grid step in real space
# x = dx * np.arange(-N, N)  # grid in real space
# dxi = np.pi / b  # Nyquist relation: grid step in Fourier space
# xi = dxi * np.arange(-N, N)  # grid in Fourier space

# # Characteristic function at time T
# m = LAMBDA * (np.exp(MU_J + 0.5 * SIGMA_J**2) - 1)  # mean of the jump size, martingale correction
# muABM = R - Q - 0.5 * SIGMA**2 - m  # drift coefficient of the arithmetic Brownian motion

# # -0.5 * (s * xi)**2 + lambda_val * (np.exp(1j * xi * alpha - 0.5 * (delta * xi)**2) - 1)
# xia = xi + 1j * ALPHA  # call
# psi = -0.5 * (SIGMA * xia)**2 + LAMBDA * (np.exp(1j * xia * ALPHA - 0.5 * (SIGMA_J * xia)**2) - 1)  # characteristic exponent
# psi_call = np.exp(psi * T)  # characteristic function
# xia = xi - 1j * ALPHA  # put
# psi = -0.5 * (SIGMA * xia)**2 + LAMBDA * (np.exp(1j * xia * ALPHA - 0.5 * (SIGMA_J * xia)**2) - 1)  # characteristic exponent
# psi_put = np.exp(psi * T)  # characteristic function

# # Fourier transform of the payoff
# b = X_WIDTH / 2  # upper bound of the support in real space
# U = S0 * np.exp(b)
# L = S0 * np.exp(-b)
# _, gc, Gc = payoff(x, xi, ALPHA, K, L, U, S0, 1)  # call
# S, gp, Gp = payoff(x, xi, -ALPHA, K, L, U, S0, 0)  # put

# # Discounted expected payoff computed with the Plancherel theorem
# c = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gc * np.conj(psi_call))))) / X_WIDTH  # call
# call_fourier = interp1d(S, c, kind='cubic')(S0)
# p = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gp * np.conj(psi_put))))) / X_WIDTH  # put
# put_fourier = interp1d(S, p, kind='cubic')(S0)

# # # ================================================================================================================
# t_fourier = time() - temp

In [35]:
# Simulation ================================================================
# # Simulate Gamma Process
# G = np.random.gamma(dt / KAPPA, KAPPA, size = (N_STEPS, N_SIMS))

# # Compute ABM increaments based on Gamma random clock
# S = THETA * G + SIGMA * np.sqrt(G) * np.random.normal(size = (N_STEPS, N_SIMS))

# # Accumulate GBM increments
# S = np.vstack((np.zeros(N_SIMS), S.cumsum(axis = 0)))
# S = S0 * np.exp(S)

In [42]:
# MONTE CARLO SOLUTION ===========================================================================================
temp = time()

call_mc_blocks = np.zeros(N_BLOCKS)
put_mc_blocks = np.zeros(N_BLOCKS)

# Monte Carlo simulation via GBM
for i in range(N_BLOCKS):
    N = np.random.poisson(LAMBDA * dt, size = (N_STEPS, N_SIMS))
    J = MU_J * N + SIGMA_J * np.sqrt(N) * np.random.randn(N_STEPS, N_SIMS)

    # Compute ABM increments and add NCPP effects
    S = (R - Q - 0.5 * SIGMA**2) * dt + SIGMA * np.sqrt(dt) * np.random.normal(size = (N_STEPS, N_SIMS))
    S += J

    # Combine to ABM
    S = np.vstack([
        np.zeros(N_SIMS),
        S.cumsum(axis = 0)
    ])
    S = S0 * np.exp(S)

    call_mc_blocks[i] = np.exp(-R * T) * np.maximum(S[-1] - K, 0).mean()
    put_mc_blocks[i] = np.exp(-R * T) * np.maximum(K - S[-1], 0).mean()
    
# Obtaining the results
call_mc = call_mc_blocks.mean()
put_mc = put_mc_blocks.mean()

# Standard error
call_mc_se = np.sqrt(call_mc_blocks.var() / N_BLOCKS)
put_mc_se = np.sqrt(put_mc_blocks.var() / N_BLOCKS)

# ================================================================================================================
t_mc = time() - temp

In [44]:
# PRINT RESULTS ===================================================================================================
print(f"{'':18s}{'Call':15s}{'Put':15s}{'CPU Time/s':15s}{'Operations':15s}")
print(f"{'Fourier':15s}{call_fourier:15.10f}{put_fourier:15.10f}{t_fourier:15.10f}{N_GRIDS:13d}")
print(f"{'Monte Carlo':15s}{call_mc:15.10f}{put_mc:15.10f}{t_mc:15.10f}{N_BLOCKS * N_SIMS:13d}")
print(f"{'MC S.E.':15s}{call_mc_se:15.10f}{put_mc_se:15.10f}")

                  Call           Put            CPU Time/s     Operations     
Fourier          22.0163676219  12.5001094255   0.0041182041          256
Monte Carlo      29.9102429527   9.1535980815  19.5395998955      2000000
MC S.E.           0.0489998571   0.0118788161
